<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração
Aqui nós extraimos os dados da internet dos carros

In [2]:
import requests
import re
import pandas as pd

from threading import Thread

## Extrair todos os cógidos FIPE

Feito manualmente, todos os códigos FIPE estão presentes na pagina:
https://www.tabelafipebrasil.com/fipe/carros
bastou fazer um regex no formato [\d]+-[\d]


arquivo gerado ficou diisponível em "https://github.com/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/FIPE-CODES.txt"

In [3]:
def get_fipe_codes():
  codes_string = requests.get("https://raw.githubusercontent.com/JoaoVitorSantiagoNogueira/case-brick-seguros/main/FIPE-CODES.txt").text
  codes_list = re.split("\r\n",codes_string)
  return codes_list

In [4]:
len(get_fipe_codes())

6893

In [5]:
get_fipe_codes()[922:933]

['004166-1',
 '004167-0',
 '004168-8',
 '004169-6',
 '004171-8',
 '004172-6',
 '004173-4',
 '004174-2',
 '004175-0',
 '004176-9',
 '004177-7']

## Pegar Informações de Veiculo

In [20]:
# https://brasilapi.com.br/api/fipe/preco/v1/038003-2  Últimos números são o código fipe

def get_json_from_fipe(Codigo_FIPE, query = ""):

  if query != "":
    query = "?tabela_referencia="+query

  response = requests.get("https://brasilapi.com.br/api/fipe/preco/v1/" + Codigo_FIPE + query)
  if response.status_code == 200:
    return response.json()


In [7]:
#sample
type(get_json_from_fipe("038003-2", "9"))

list

#Cria a tabela inteira de um veiculo
Navegando preenchendo mês-a-mês, no formato mm-aa

## Cria o data frame base

In [8]:
df = pd.json_normalize(get_json_from_fipe("038003-2", "10"))
df

,valor,marca,modelo,anoModelo,combustivel,codigoFipe,mesReferencia,tipoVeiculo,siglaCombustivel,dataConsulta
0,"R$ 18.742,00",Acura,Integra GS 1.8,1992,Gasolina,038003-2,março de 2006,1,G,"segunda-feira, 13 de maio de 2024 18:54"
1,"R$ 17.197,00",Acura,Integra GS 1.8,1991,Gasolina,038003-2,março de 2006,1,G,"segunda-feira, 13 de maio de 2024 18:54"


In [9]:
def generate_full_fipe_entry(codigo_fipe, resultados, numero_tread):
    new_line = get_json_from_fipe(codigo_fipe)
    #completar com outros anos de dados da tabela fipe
    print("tread " + str(codigo_fipe) + " has started")
    i=0
    entrada_atual = new_line[0]["mesReferencia"]
    ultima_entrada = None
    while (ultima_entrada != entrada_atual):
      i += 1
      serie_t.status_code == 200:
      if type(serie_temporal) == list :
        ultima_entrada = serie_temporal[0]["mesReferencia"]
        for modelo_atual in new_line:
          for modelo_antigo in serie_temporal:
            if (modelo_atual["anoModelo"] == modelo_antigo["anoModelo"] and
               modelo_atual["siglaCombustivel"] == modelo_antigo["siglaCombustivel"]):
               modelo_atual[modelo_antigo["mesReferencia"]] = modelo_antigo["valor"]
               break

    resultados [numero_tread] = new_line
    print("tread " + str(codigo_fipe) + " is done")
    return

In [10]:
#processo demorado, em torno de 1 minuto por veiculo
def create_whole_dataset(codigos, num_treads = 10):
  #criar um dataframe e atualizar eles é custoso
  #https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-and-then-filling-it
  #fazemos primeiro em dicts e depois criamos o data frame
  data_set = []

  threads = [None] * num_treads
  results = [None] * num_treads

  offset= 0 #usado para poder loopar o numero de treads

  created_thread_count = 0
  for i in range(len(threads)): ## cada linha gera sua pesquisa própria
    if i+offset >= len (codigos):
      break
    created_thread_count += 1
    threads[i] = Thread(target = generate_full_fipe_entry, args = (codigos[i+offset], results, i))
    threads[i].start()


  for i in range(created_thread_count):
    threads[i].join()

  return results

In [11]:
result = create_whole_dataset(['004166-1',
 '004167-0',
 '004168-8',
 '004169-6',
 '004171-8',
 '004172-6',
 '004173-4',
 '004174-2',
 '004175-0',
 '004176-9',
 '004177-7'
])
df = pd.json_normalize(result)
df

KeyboardInterrupt: 

In [12]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
def ds_normalizer():
  r = []
  for item in result:
    r += item
  df = pd.json_normalize(r)
  return df

In [14]:
def save_csv_to_drive(csv, filename):
  csv.to_csv('/content/drive/My Drive/' + filename + ".csv", index= False)

In [ ]:
serializar código de forma melhor
https://stackoverflow.com/questions/40391898/send-simultaneous-requests-python-all-at-once

In [22]:
#criar tabelas com força bruta

cods = get_fipe_codes()

i=0
while i < len(cods):
  if len(cods) < i+10:
    offset = len(cods) - i
  else:
    offset=50
  dt = create_whole_dataset(cods[i:i+offset], num_treads = offset)
  dt = ds_normalizer(dt)
  save_csv_to_drive(dt, str(i)+"to"+str(i+offset))
  i+=offset




tread 038002-4 has started
tread 038001-6 has started
tread 060002-4 has startedtread 060001-6 has started

tread 038003-2 has started
tread 060004-0 has startedtread 060005-9 has started
tread 060003-2 has startedtread 060007-5 has started
tread 060006-7 has started


tread 060008-3 has started
tread 060010-5 has started
tread 060009-1 has startedtread 060015-6 has started

tread 060016-4 has startedtread 060013-0 has started
tread 006001-1 has started
tread 060012-1 has started

tread 060014-8 has started
tread 060011-3 has startedtread 006005-4 has started

tread 006008-9 has started
tread 006002-0 has startedtread 006004-6 has started
tread 006003-8 has started

tread 006006-2 has started
tread 006007-0 has started
tread 006010-0 has started
tread 006011-9 has started
tread 006013-5 has started
tread 006014-3 has startedtread 006009-7 has started

tread 006016-0 has startedtread 006015-1 has started

tread 037001-0 has started
tread 037002-9 has started
tread 006017-8 has started
t

Exception in thread Thread-265 (generate_full_fipe_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 287, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Tr

KeyboardInterrupt: 